In [6]:
from quilt.data.andrewbrown import invasive

In [2]:
invasive.sample_submission().head()

,name,invasive
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5


In [3]:
import utils

Using Theano backend.


In [4]:
from vgg16 import Vgg16

In [5]:
batch_size = 64